# Binary Trees
Formally, a binary tree is either empty, or a *root* node $r$ together with a left binary tree and a right binary tree. The subtrees themselves are binary trees.

### Tips
- Learn to recognize when the stack **LIFO** property is **applicable**. For example, **parsing** typically benefits from a stack.
- Consider **augmenting** the basic stack or queue data structure to support additional operations, such as finding the maximum element.

In [61]:
from typing import Deque, Iterator, List, Optional
from collections import deque, namedtuple
from dataclasses import dataclass

from utils import run_tests

### Class

In [62]:
class BinaryTreeNode:

    def __init__(self, data=None, left=None, right=None) -> None:
        self.data = data
        self.left = left 
        self.right = right 


def tree_traversal_preorder(root: BinaryTreeNode) -> None:
    if root:
        print(root.data, end=' ')
        tree_traversal_preorder(root.left)
        tree_traversal_preorder(root.right)

def tree_traversal_inorder(root: BinaryTreeNode) -> None:
    if root:
        tree_traversal_inorder(root.left)
        print(root.data, end=' ')
        tree_traversal_inorder(root.right)

def tree_traversal_postorder(root: BinaryTreeNode) -> None:
    if root:
        tree_traversal_postorder(root.left)
        tree_traversal_postorder(root.right)
        print(root.data, end=' ')


def tree_traversal_levelorder(root: BinaryTreeNode) -> None:
    '''
    use a queue
    '''
    if root is None:
        return 

    queue = Deque()
    queue.append(root)

    while queue:
        node = queue.popleft()
        print(node.data, end=' ')

        # add children to queue
        if node.left:
            queue.append(node.left)
        if node.right:
            queue.append(node.right)



def tree_height(root: BinaryTreeNode) -> int:
    if root is None:
        return 0

    # depth first search/post order
    return 1 + max([tree_height(root.left), tree_height(root.right)])


def size(root: BinaryTreeNode) -> int:
    if root is None:
        return 0
    
    return 1 + size(root.left) + size(root.right)
    

def tree_insert(root: BinaryTreeNode, data) -> None:
    node = BinaryTreeNode(data)

    if root is None:
        root = node
        return

    # level-order traversal
    q = Deque()
    q.append(root)

    while q:
        temp = q.popleft()

        if temp.left:
            q.append(temp.left)
        else:
            temp.left = node 
            return

        if temp.right:
            q.append(temp.right)
        else:
            temp.right = node 
            return


In [63]:
def make_tree_example() -> BinaryTreeNode:
    tree = BinaryTreeNode(314,
        left=BinaryTreeNode(6,
            left=BinaryTreeNode(271,
                left=BinaryTreeNode(28),
                right=BinaryTreeNode(0)
                ),
            right=BinaryTreeNode(561,
                left=None,
                right=BinaryTreeNode(3,
                    left=BinaryTreeNode(17),
                    right=None
                )
            )
        ),
        right=BinaryTreeNode(7,
            left=BinaryTreeNode(2,
                left=None,
                right=BinaryTreeNode(1,
                    left=BinaryTreeNode(401,
                        left=None,
                        right=BinaryTreeNode(641)
                    ),
                    right=BinaryTreeNode(257)
                )
            ),
            right=BinaryTreeNode(272,
                left=None,
                right=BinaryTreeNode(29)
            )
        )
    )
    return tree

tree = make_tree_example()

In [64]:
# Pre-order
tree_traversal_preorder(tree)

314 6 271 28 0 561 3 17 7 2 1 401 641 257 272 29 

In [65]:
# In Order
tree_traversal_inorder(tree)

28 271 0 6 561 17 3 314 2 401 641 1 257 7 272 29 

In [66]:
# post-order
tree_traversal_postorder(tree)

28 0 271 17 3 561 6 641 401 257 1 2 29 272 7 314 

In [67]:
tree_height(tree)

6

In [68]:
size(tree)

16

In [69]:
tree_traversal_levelorder(tree)

314 6 7 271 561 2 272 28 0 3 1 29 17 401 257 641 

In [70]:
tree_insert(tree, 1000)
tree_traversal_levelorder(tree)

314 6 7 271 561 2 272 28 0 1000 3 1 29 17 401 257 641 

In [71]:
tree = make_tree_example()

### 9.1: Test if a Binary Tree is Height Balanced

In [72]:
def is_height_balanced(root: BinaryTreeNode) -> bool:

    # base condition
    if root is None:
        return True
    
    # get tree height of left and right subtree
    left_height, right_height = tree_height(root.left), tree_height(root.right)
    if abs(left_height - right_height) <= 1 and is_height_balanced(root.left) and is_height_balanced(root.right): 
        return True

    return False
    

Time complxity is $O(n)$ (same as post-order traversal) and space complexity is $O(h)$

In [73]:
is_height_balanced(tree)

False

In [74]:
tree_balanced = BinaryTreeNode(4, left=BinaryTreeNode(2), right=BinaryTreeNode(1))
is_height_balanced(tree_balanced)

True

### 9.2: Test if a Binary Tree is Symmetric

In [75]:
def is_symmetric(tree: BinaryTreeNode) -> bool:

    def is_symmetric_helper(sub_tree0: BinaryTreeNode, sub_tree1: BinaryTreeNode) -> bool:
        if not sub_tree0 and not sub_tree1:
            return True
        elif sub_tree0 and sub_tree1:
            return (sub_tree0.data == sub_tree1.data and
                    is_symmetric_helper(sub_tree0.left, sub_tree1.right) and
                    is_symmetric_helper(sub_tree0.right, sub_tree1.left)
                    )
        # one subtree is empty and the other is not
        else:
            return False
    return not tree or is_symmetric_helper(tree.left, tree.right)

Time and space complexity is $O(n)$ and $O(h)$ respectively

In [76]:
is_symmetric(tree)

False

In [77]:
is_symmetric(BinaryTreeNode(5, 
                left=BinaryTreeNode(10, 
                    left=None, 
                    right=BinaryTreeNode(100)
                ),
                right=BinaryTreeNode(10,
                    left=BinaryTreeNode(100),
                    right=None
                )
            )
        )
    

True

### 9.3: Find the Least Common Ancestor of Two Nodes

### 9.4: Find the Least Common Ancestor of Two Nodes with Parent Nodes

In [78]:
class BinaryTreeNodeParent:

    def __init__(self, data=None, left=None, right=None, parent=None) -> None:
        self.data = data
        self.left = left 
        self.right = right 
        self.parent = parent


In [79]:
root = BinaryTreeNodeParent(data=100)
ln = BinaryTreeNodeParent(parent=root, data=10)
rn = BinaryTreeNodeParent(parent=root, data=15)
root.left, root.right = ln, rn
lln = BinaryTreeNodeParent(data=4, parent=ln)



In [80]:
# walk up tree until get to lca
def lca_parent(node0: BinaryTreeNodeParent, node1: BinaryTreeNodeParent) -> BinaryTreeNodeParent:

    # calculate depth of each node; if there is a difference, need to walk up longer one
    def get_depth(node: BinaryTreeNodeParent) -> int:
        depth = 0
        while node.parent:
            depth += 1
            node = node.parent
        return depth 
    
    depth_node0, depth_node1 = get_depth(node0), get_depth(node1)
    # make node0 longer node to simplify code
    if depth_node1 > depth_node0:
        node0, node1 = node1, node0 
    
    depth_diff = abs(depth_node0 - depth_node1)
    while depth_diff:
        node0 = node0.parent
        depth_diff -= 1
    
    # walk up tree til nodes same
    while node0 is not node1:
        node0, node1 = node0.parent, node1.parent
    
    return node0

The time and space complexity are that of computing the depth, namely $O(h)$ and $O(1)$, respectively

In [81]:
lca_parent(lln, rn).data

100

### 9.5: Sum the Root-to-Leaf Paths in a Binary Tree

In [82]:
def sum_root_to_leaf(tree: BinaryTreeNode) -> int:

    def helper(tree: BinaryTreeNode, partial_sum: int=0):
        if not tree:
            return 0
        
        partial_sum = partial_sum * 2 + tree.data 

        if not tree.left and not tree.right:  # leaf
            return partial_sum
        else:   # non-leaf
            return helper(tree.left, partial_sum) + helper(tree.right, partial_sum)
    
    return helper(tree)

In [83]:
binary_tree = BinaryTreeNode(1,
                left=BinaryTreeNode(0,
                    left=BinaryTreeNode(0,
                        left=BinaryTreeNode(0),
                        right=BinaryTreeNode(1)
                        ),
                    right=BinaryTreeNode(1,
                        left=None,
                        right=BinaryTreeNode(1,
                            left=BinaryTreeNode(0),
                            right=None
                            )
                        )
                    ),
                right=BinaryTreeNode(1,
                    left=BinaryTreeNode(0,
                        left=None,
                        right=BinaryTreeNode(0,
                            left=BinaryTreeNode(1,
                                left=None,
                                right=BinaryTreeNode(1)
                                ),
                            right=BinaryTreeNode(0)
                            )
                        ),
                    right=BinaryTreeNode(0,
                        left=None,
                        right=BinaryTreeNode(0))
                    )
                )
tree_traversal_levelorder(binary_tree)

1 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 

In [84]:
sum_root_to_leaf(binary_tree)

126

Time and space complexity is $O(n)$ and $O(h)$ respectively

### 9.6: Find a Root to Leaf Path with Specified Sum

In [85]:
def has_path_sum(tree: BinaryTreeNode, remaining_weight: int) -> bool:
    if not tree:
        return None

    if not tree.left and not tree.right:   # leaf
        return remaining_weight == tree.data

    # non-leaf 
    return has_path_sum(tree.left, remaining_weight - tree.data) or has_path_sum(tree.right, remaining_weight - tree.data)

Time and space complexity is $O(n)$ and $O(h)$ respectively

In [86]:
has_path_sum(tree, 591)

True

### 9.7: Implement an Inorder Traversal without Recursion

In [87]:
def inorder_traversal_no_recursion(tree: BinaryTreeNode) -> List[int]:
    result = []

    if not tree:
        return result

    in_process = [(tree, False)]   # used as stack
    while in_process:
        node, left_subtree_traversed = in_process.pop()
        if node:
            if left_subtree_traversed:
                result.append(node.data)
            else:
                # process left subtree, print root, process right subtree
                in_process.append((node.right, False))
                in_process.append((node, True))
                in_process.append((node.left, False))
    
    return result

Time and space complexity is $O(n)$ and $O(h)$ respectively

In [88]:
inorder_traversal_no_recursion(tree)

[28, 271, 0, 6, 561, 17, 3, 314, 2, 401, 641, 1, 257, 7, 272, 29]

#### Variant: Postorder Traversal without Recursion

In [89]:
def postorder_traversal_no_recursion(tree: BinaryTreeNode) -> List[int]:
    result = []

    if not tree:
        return result

    in_process = [(tree, False)]    # used as stack
    while in_process:
        node, left_subtree_traversed = in_process.pop()
        if node:
            if left_subtree_traversed:
                result.append(node.data)
            else:
                # process left subtree, right subtree, print root
                in_process.append((node, True))
                in_process.append((node.right, False))
                in_process.append((node.left, False))
    
    return result

In [90]:
postorder_traversal_no_recursion(tree)

[28, 0, 271, 17, 3, 561, 6, 641, 401, 257, 1, 2, 29, 272, 7, 314]

#### Variant: Preorder Traversal without Recursion

In [91]:
def preorder_traversal_no_recursion(tree: BinaryTreeNode) -> List[int]:
    result = []

    if not tree:
        return result

    in_process = [(tree, False)]    # used as stack
    while in_process:
        node, left_subtree_traversed = in_process.pop()
        if node:
            if left_subtree_traversed:
                result.append(node.data)
            else:
                # print node, process left subtree, right subtree
                in_process.append((node.right, False))
                in_process.append((node.left, False))
                in_process.append((node, True))
    
    return result

In [92]:
preorder_traversal_no_recursion(tree)

[314, 6, 271, 28, 0, 561, 3, 17, 7, 2, 1, 401, 641, 257, 272, 29]

### 9.8: Find Kth Node in Binary Tree
Kth node in an inorder traversal   
Assume you have size of tree rooted at particular node

In [93]:
def add_size(tree: BinaryTreeNode) -> None:
    if not tree:
        return

    tree.size = size(tree)
    add_size(tree.left)
    add_size(tree.right)

add_size(tree)

In [94]:
# if k is greater than the number of nodes in the left subtree, k cannot lie in the left subtree
# if the left subtree contains L nodes, then kth node in the original tree is the (k-L)th node when skipping the left subtree
def find_kth_node(tree: BinaryTreeNode, k: int) -> Optional[BinaryTreeNode]:
    
    while tree:
        left_size = tree.left.size if tree.left else 0
        if left_size + 1 < k:    # kth node in right sub tree
            k -= left_size + 1   # + 1 for root node 
            tree = tree.right 
        elif left_size == k - 1: # kth node
            return tree
        else:                    # kth node in left subtree
            tree = tree.left
    
    return None  # kth unreachable

Time complexity is $O(h)$

In [95]:
tree_traversal_inorder(tree)
print()
print(find_kth_node(tree, 3).data)
print(find_kth_node(tree, 10).data)

28 271 0 6 561 17 3 314 2 401 641 1 257 7 272 29 
0
401


### 9.9: Compute Successor